In [ ]:
# !pip install minsearch

In [2]:
import minsearch

In [51]:
import json
import pprint

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [8]:
q = 'the course has already started, can I still enroll?'

In [9]:
index.fit(documents)

In [17]:
!pip install dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [dotenv]


In [ ]:
import os
from groq import Groq
from dotenv import load_dotenv

# Завантаження змінних середовища з файлу .env
load_dotenv()

In [38]:
# Убедитесь, что переменная окружения GROQ_API_KEY установлена
api_key = os.environ.get("GROQ_API_KEY")

if api_key is None:
    raise ValueError("API key is not set. Please set the GROQ_API_KEY environment variable.")

client = Groq(api_key=api_key)

In [39]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Fast language models have become increasingly important in recent years due to their ability to process and generate human-like text quickly and efficiently. Here are some reasons why fast language models are significant:

1. **Improved user experience**: Fast language models enable applications to respond quickly to user input, providing a more seamless and engaging experience. For example, a language-powered chatbot can respond in real-time to user queries, making it feel more natural and human-like.
2. **Increased efficiency**: By processing text faster, language models can help reduce the time and resources required to complete tasks, such as language translation, text summarization, and content creation.
3. **Scalability**: Fast language models can handle large volumes of text data, making them suitable for applications that require processing vast amounts of information, such as search engines, social media platforms, and customer service chatbots.
4. **Multi-lingual support**: F

In [ ]:
from openai import OpenAI

api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

In [40]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [57]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION. Translate the answer into Russian.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [58]:
def llm(prompt):
    response = client.chat.completions.create(
        model='llama3-8b-8192',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [14]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [59]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [60]:
print(rag(query))

Given the context, here's the answer to the question "How do I run Kafka?":

answer:Unfortunately, the FAQ database does not provide a general answer on how to run Kafka. However, it seems that the focus is on running Kafka-related applications, such as producers, consumers, and KStreams, using Java or Python.

For Java, you can run a Kafka producer by navigating to the project directory and executing the following command:

java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java

For Python, you need to create a virtual environment using the command `python -m venv env`, install the necessary packages using `pip install -r ../requirements.txt`, and then activate the environment using `source env/bin/activate` (or `env/Scripts/activate` on Windows). Once the environment is activated, you can run your Python Kafka application.

Please note that the FAQ database does not provide information on how to install and run a standalone Kafka instance. If 

In [61]:
print(rag('the course has already started, can I still enroll?'))

 QUESTION: the course has already started, can I still enroll?

ANSWER: Yes, even if you don't register, you're still eligible to submit the homeworks.

TRANSLATION IN RUSSIAN:
Да, даже если вы не зарегистрировались, вы по-прежнему можете отправлять домашние задания.


In [62]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [69]:
!pip install elastic-transport

In [70]:
from elasticsearch import Elasticsearch

In [126]:
# Убедитесь, что переменная окружения ES_API_KEY установлена
load_dotenv()
api_key = os.environ.get("ESAPI_KEY")
cloud_id = os.environ.get("ES_ID")
api_key

'QmRWbWlaY0JyWFhWMy1Wd3dkVFk6aUlKNUQyQ1lOQXcwTUlOcTNyNkRnQQ=='

In [138]:
!curl -H "Authorization: ApiKey QmRWbWlaY0JyWFhWMy1Wd3dkVFk6aUlKNUQyQ1lOQXcwTUlOcTNyNkRnQQ==" \
     https://my-deployment-c38b3f.es.us-east-2.aws.elastic-cloud.com


{
  "name" : "instance-0000000001",
  "cluster_name" : "e98150c8fbe6463db33d9a402393966f",
  "cluster_uuid" : "XcNYcAI9RjGXz1-iC4Gccw",
  "version" : {
    "number" : "9.0.2",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "0a58bc1dc7a4ae5412db66624aab968370bd44ce",
    "build_date" : "2025-05-28T10:06:37.834829258Z",
    "build_snapshot" : false,
    "lucene_version" : "10.1.0",
    "minimum_wire_compatibility_version" : "8.18.0",
    "minimum_index_compatibility_version" : "8.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [2]:
!curl localhost:9200

{
  "name" : "991c038d2dea",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "8oVckcnxRqa5CNFT0-tMxQ",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [127]:
es_client = Elasticsearch(
    cloud_id=cloud_id,
    api_key=api_key
)

In [128]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [129]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [130]:
!pip install tqdm

In [131]:
from tqdm.auto import tqdm

/home/oleg_yamgurov/llm-zoomcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [132]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [01:43<00:00,  9.16it/s]


In [133]:
query = 'I just disovered the course. Can I still join it?'

In [134]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [135]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [136]:
rag(query)

"QUESTION: I just discovered the course. Can I still join it?\n\nANSWER: Yes, even if you don't register, you're still eligible to submit the homeworks.\n\nTRANSLATION INTO RUSSIAN:\n\nВопрос: Я только что обнаружил курс. Могу я все еще присоединиться к нему?\n\nОтвет: Да, даже если вы не зарегистрируетесь, вы все еще можете сдавать домашние задания.\n\nNote: The answer is based on the CONTEXT provided, which states that even if you don't register, you're still eligible to submit homeworks."

In [147]:
query = 'How do execute a command on a Kubernetes pod?'

In [156]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },

            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return response['hits']['hits']

elastic_search(query)

[{'_index': 'course-questions',
  '_id': 'jkpqiZcBfjM-EchMzfrW',
  '_score': 44.50556,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'HUpriZcBfjM-EchMC_sQ',
  '_score': 35.433445,
  '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
   'section': '10. Kubernetes and TensorFlow Serving',
   'question': 'Kubernetes-dashboard',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'rkpqiZcBfjM-EchM2_qM',
  '_score': 33.70974,
  '_source': {'text': 'You 

In [158]:
query = 'How do copy a file to a Docker container?'

In [159]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return response['hits']['hits'][0]['_score'], result_docs

max_score, res_docs = elastic_search(query)

In [162]:
res_docs[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

In [163]:
query="How do copy a file to a Docker container?"

In [164]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"    
        
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [165]:
prompt = build_prompt(query, res_docs)

print("Length of the prompt: ", len(prompt))

Length of the prompt:  1447


In [171]:
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do copy a file to a Docker container?\n\nCONTEXT: \nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory containe

In [168]:
import tiktoken

In [169]:
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens=encoding.encode(prompt)
print('Num tokens: ', len(tokens))

Num tokens:  321


In [173]:
encoding.decode_single_token_bytes(63842)

b"You're"

In [178]:
output = llm(prompt)
print(output)

Based on the context, to copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is as follows:

`docker cp /path/to/local/file_or_directory container_id:/path/in/container`

Replace `/path/to/local/file_or_directory` with the path to the file you want to copy, and `container_id` with the ID of the container you want to copy the file to.


In [183]:
# Прайсы за 1М токенов ($)
MODEL_PRICING = {
    "gpt-4.1": {"prompt": 0.0020, "cached": 0.0005, "output": 0.0080},
    "gpt-4.1-mini": {"prompt": 0.0004, "cached": 0.0001, "output": 0.0016},
    "gpt-4.1-nano": {"prompt": 0.0001, "cached": 0.000025, "output": 0.0004},
    "openai-o3": {"prompt": 0.0020, "cached": 0.0005, "output": 0.0080},
    "openai-o4-mini": {"prompt": 0.0011, "cached": 0.000275, "output": 0.0044}
}

def estimate_cost(prompt: str, output: str, model: str = "gpt-4.1", discount: float = 0.0, cached_input: bool = False):
    if model not in MODEL_PRICING:
        raise ValueError(f"Модель '{model}' не найдена в прайс-листе")

    pricing = MODEL_PRICING[model]
    encoding = tiktoken.encoding_for_model("gpt-4o")

    # Подсчёт токенов
    prompt_tokens = len(encoding.encode(prompt))
    output_tokens = len(encoding.encode(output))
    total_tokens = prompt_tokens + output_tokens

    # Выбор тарифа для prompt
    prompt_rate = pricing["cached"] if cached_input and "cached" in pricing else pricing["prompt"]

    # Расчёт стоимости
    raw_prompt_cost = (prompt_tokens / 1_000) * prompt_rate
    raw_output_cost = (output_tokens / 1_000) * pricing["output"]
    total_raw_cost = raw_prompt_cost + raw_output_cost
    total_discounted = total_raw_cost * (1 - discount)

    return {
        "model": model,
        "cached_input": cached_input,
        "discount": discount,
        "prompt_tokens": prompt_tokens,
        "output_tokens": output_tokens,
        "total_tokens": total_tokens,
        "cost_prompt": round(raw_prompt_cost, 6),
        "cost_output": round(raw_output_cost, 6),
        "total_cost": round(total_discounted, 6)
    }


In [184]:
res = estimate_cost(
    prompt=prompt,
    output=output,
    model="gpt-4.1-nano",
    cached_input=True,
    discount=0.5
)

for k, v in res.items():
    print(f"{k}: {v}")


model: gpt-4.1-nano
cached_input: True
discount: 0.5
prompt_tokens: 321
output_tokens: 87
total_tokens: 408
cost_prompt: 8e-06
cost_output: 3.5e-05
total_cost: 2.1e-05
